In [1]:
import pandas as pd
import numpy as np
import sklearn as sklearn

data = pd.read_csv("Breast_Cancer.csv", dtype={'Status': 'category'})

In [ ]:
data.head()

In [2]:
df = pd.DataFrame(data)

In [4]:
# OneHot Encoding to convert to float - MARITAL STATUS
maritalstatus_encode = 'Marital Status'

maritalstatus_encoded = pd.get_dummies(df[maritalstatus_encode], prefix = maritalstatus_encode)

df = pd.concat([df, maritalstatus_encoded], axis=1)

df.drop('Marital Status', axis=1, inplace=True)

In [5]:
# OneHot Encoding to convert to float - MARITAL STATUS
race_encode = 'Race'

race_encoded = pd.get_dummies(df[race_encode], prefix = race_encode)

df = pd.concat([df, race_encoded], axis=1)

df.drop('Race', axis=1, inplace=True)

In [10]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

X = pd.concat([df[['Age']], maritalstatus_encoded, race_encoded], axis=1)
Y = df[['Tumor Size']]

X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=1234)

In [11]:
model = DecisionTreeClassifier()
model.fit(X_train, y_train)

DecisionTreeClassifier()

In [12]:
predictions = model.predict(X_test)
print(predictions[:5])

[16 30 25 15 25]


In [13]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, predictions))

0.05765407554671968


5% correct